In [1]:
import pandas as pd
import numpy as np
import time
import urllib.request
import os
import pickle
from os import path

In [61]:
import torch
torch.cuda.is_available()

True

In [94]:
import torch

torch.cuda.get_device_name()

'NVIDIA GeForce RTX 3060'

In [92]:
from torch import cuda
assert cuda.is_available()
assert cuda.device_count() > 0
print(cuda.get_device_name(cuda.current_device()))

NVIDIA GeForce RTX 3060


In [95]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification,pipeline

tokenizer = AutoTokenizer.from_pretrained('snunlp/KR-FinBert-SC')
model=AutoModelForSequenceClassification.from_pretrained('snunlp/KR-FinBert-SC')

senti_classifier = pipeline('text-classification',model=model,tokenizer=tokenizer,device=0)

In [96]:
def allsenticlassifier(allnewsdf,day):
    newsdf=(allnewsdf.loc[(allnewsdf['날짜']==day)])[['내용']]
    newslist=newsdf['내용'].values.tolist()
    x = newslist
    y = [x['label'] for x in senti_classifier(x)]
   
    
    neutralC = y.count('neutral')
    negativeC = y.count('negative')
    positiveC = y.count('positive')
    
    addC=neutralC+negativeC+positiveC
    
    neutralP=neutralC/addC
    negativeP=negativeC/addC
    positiveP=positiveC/addC
    
    return neutralP,negativeP,positiveP


In [97]:
Kospi_50 = pd.DataFrame({'종목코드':['005930', '000660', '207940', '051910', '006400'
                                    , '005380', '000270', '207940', '005490', '035420'
                                    , '003670', '035720', '068270', '012330', '066570'
                                    , '028260', '105560', '055550', '096770', '032830'
                                    , '034730', '086790', '015760', '034020', '033780'
                                    , '000810', '009150', '017670', '010130', '018260'
                                    , '011200', '138040', '316140', '010950', '051900'
                                    , '009830', '003490', '024110', '030200', '011170'
                                    , '011070', '009540', '361610', '036570', '086280'
                                    , '047050', '090430', '241560', '034220','010140'],
                             '종목명':['삼성전자', 'SK하이닉스', '삼성바이오로직스', 'LG화학', '삼성SDI'
                                   , '현대차', '기아', '삼성바이오로직스', 'POSCO홀딩스', 'NAVER'
                                   , '포스코퓨처엠', '카카오', '셀트리온', '현대모비스', 'LG전자'
                                   , '삼성물산', 'KB금융', '신한지주', 'SK이노베이션', '삼성생명'
                                   , 'SK', '하나금융지주', '한국전력', '두산에너빌리티', 'KT&G'
                                   , '삼성화재', '삼성전기', 'SK텔레콤', '고려아연', '삼성에스디에스'
                                   , 'HMM', '메리츠금융지주', '우리금융지주', 'S-Oil', 'LG생활건강'
                                   , '한화솔루션', '대한항공', '기업은행', 'KT', '롯데케미칼'
                                   , 'LG이노텍', 'HD한국조선해양', 'SK아이이테크놀로지', '엔씨소프트', '현대글로비스'
                                   , '포스코인터내셔널', '아모레퍼시픽', '두산밥캣','LG디스플레이','삼성중공업']})

In [105]:
def storkbart(company):
    allcsvnum=0
    while True:
        alldf=pd.DataFrame()
        try:
            for csvnum in range(0,50):
                nowdf = pd.read_csv(f'E:\stock/주토방/{company}{allcsvnum*100}.csv', encoding='euc-kr',names=['날짜','내용'])
                alldf = pd.concat([alldf,nowdf], ignore_index=True)
                allcsvnum+=1
        except FileNotFoundError as e:
            pass

        print('---------------------')

        alldf['날짜']=alldf['날짜'].str.replace('.','')
        alldf['날짜']=pd.to_numeric(alldf['날짜'])

        alldf=alldf[alldf['날짜']<20230912]


        print(len(alldf))

        if len(alldf)==0:
            print('없음')
            continue

        alldf=alldf.sort_values(by=['날짜'])
        alldf['글자수']=alldf['내용'].str.count('')-1
        alldf=alldf.loc[alldf['글자수']>2]
        alldf=alldf.loc[alldf['글자수']<512]

        df_day_list=alldf['날짜'].to_list()
        df_day_setint=set(map(int,df_day_list))
        df_day_listint = list(df_day_setint)
        df_day_listint.sort()



        for day in df_day_listint:
            neutralP,negativeP,positiveP=allsenticlassifier(alldf,day)
            print(day)
            f = open(f'E:\stock/감성분석/{company}.csv', 'w')
            f.write(str(day)+','+str(neutralP)+','+str(negativeP)+','+str(positiveP))
            f.close()

In [106]:
for i in range(0,1):
    storkbart(Kospi_50.iloc[i,1])

---------------------
1877
20230910


KeyboardInterrupt: 

'삼성전자'

In [54]:
alldf=pd.DataFrame()
allcsvnum=0
try:
    for csvnum in range(1,11):
        nowdf = pd.read_csv(f'E:\stock/주토방/삼성전자{allcsvnum*100}.csv', encoding='euc-kr',names=['날짜','내용'])
        alldf = pd.concat([alldf,nowdf], ignore_index=True)
        print(allcsvnum)
        allcsvnum+=1
except FileNotFoundError as e:
    print(allcsvnum)
    pass

print('---------------------')

alldf['날짜']=alldf['날짜'].str.replace('.','')
alldf['날짜']=pd.to_numeric(alldf['날짜'])

alldf=alldf[alldf['날짜']<20230915]


print(len(alldf))


alldf=alldf.sort_values(by=['날짜'])
alldf['글자수']=alldf['내용'].str.count('')-1
alldf=alldf.loc[alldf['글자수']>2]
alldf=alldf.loc[alldf['글자수']<512]

0
1
2
3
4
5
6
7
8
9
---------------------
1000


In [55]:
alldf

,날짜,내용,글자수
999,20230913,용산돼지가 누구인지 아는거냐,16
964,20230913,이재명 대통령 만들자. 밀약있었다.,20
963,20230913,낼 삼전도 올라가겠군,11
962,20230913,일본 본토에서 멀다고 우리나라에 훨씬 더 가까운 대마도에 후쿠시마핵폐기물...,116
961,20230913,2찍 개 돼지들이 광화문에 있으므로 잠 못 자는데 철거 할만 하지 용산총독부,49
...,...,...,...
337,20230914,일본지자체통과! 부산에서 27km 거리! 이건 한국정부 협의 사항일까? ...,125
338,20230914,나름 26일 전후로 7만8~9천 예상해봅니다,24
339,20230914,니들끼리 잔치허냐? 엉? 크크루루룰라 크르큭큭큭~,29
327,20230914,공감 돌려가면서 잘한다아 너는 네 가족이 학벌을 위조했다며 조사당하면 좋냐? ...,459


In [98]:
%%time
df_day_list=alldf['날짜'].to_list()
df_day_setint=set(map(int,df_day_list))
df_day_listint = list(df_day_setint)
df_day_listint.sort()

print(df_day_listint)

for day in df_day_listint:
    neutralP,negativeP,positiveP=allsenticlassifier(alldf,day)
    print(day)
    f = open(f'E:\stock/감성분석/삼성전자2.csv', 'w')
    f.write(str(day)+','+str(neutralP)+','+str(negativeP)+','+str(positiveP))
    f.close()

[20230913, 20230914]
20230913
20230914
CPU times: total: 13.5 s
Wall time: 13.6 s


In [76]:
watch -n -1 nvidia-smi

SyntaxError: invalid syntax (590320565.py, line 1)

In [14]:
df_day_list=alldf['날짜'].to_list()
df_day_setint=set(map(int,df_day_list))
df_day_listint = list(df_day_setint)
df_day_listint.sort()

In [15]:
df_day_listint

[20230914]

In [ ]:
negativeP

In [ ]:
positiveP

In [ ]:
f = open(f'E:\stock/감성분석/삼성전자.csv', 'w')

In [ ]:
f.write(str(day)+','+str(neutralP)+','+str(negativeP)+','+str(positiveP))

In [ ]:
for day in df_day_listint:
    neutralP,negativeP,positiveP=allsenticlassifier(alldf,day)
    print(day)

In [ ]:
df = pd.read_csv('./SK하이닉스100000.csv', encoding='euc-kr',names=['날짜','내용'])

In [ ]:
alldf['날짜'].tail(1)

In [ ]:
df['날짜']=df['날짜'].str.replace('.','')
df['날짜']=pd.to_numeric(df['날짜'])
df=df.sort_values(by=['날짜'])
df['글자수']=df['내용'].str.count('')-1
df=df.loc[df['글자수']>2]
df=df.loc[df['글자수']<1000]